In [1]:
import pandas as pd
from IPython.display import display, Markdown, Latex 
from pathlib import Path
import sys
import csv
import pandas as pd
import numpy as np
import os
from os import path
from datetime import datetime
from urllib.parse import urlparse, urljoin
import requests
import ntpath
from bs4 import BeautifulSoup, Comment
import time
from pprint import pprint 
import shutil
#!pip install tldextract
import tldextract
import multiprocessing
from multiprocessing import Process, Queue, cpu_count
import os
from pprint import pprint
from os import path, listdir 
pd.options.display.float_format = '{:,.2f}'.format 
pd.set_option('display.max_colwidth', -1)


In [2]:
display(Markdown("""
# System Info: 
"""))
import psutil
from multiprocessing import Process, Queue, cpu_count
from pprint import pprint
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory() 
# you can convert that object to a dictionary 
pprint(dict(psutil.virtual_memory()._asdict()))
print('\n#CPUs: {}'.format(cpu_count()))


# System Info: 


{'active': 3379728384,
 'available': 3392143360,
 'free': 14622720,
 'inactive': 3071213568,
 'percent': 60.5,
 'total': 8589934592,
 'used': 4917534720,
 'wired': 1537806336}

#CPUs: 4


In [3]:
display(Markdown("""
# Load Module and Build Batch: 
""")) 
sys.path.insert(0, os.path.abspath('C:/Coding/Get'))
import CollegeContent as CC
#url='https://www.university-list.net/us/universities-2000.htm'


# Load Module and Build Batch: 


ModuleNotFoundError: No module named 'CollegeContent'

In [ ]:
'''
url='http://www.atsu.edu'
name='A.T. Still University of Health Sciences'
filename= 'A_100000.csv',
urlfile='A_100000url.csv'  
c=CC.CollegeCrawl(name, url,urlfile, filename, _gapInSecond=0, _maxPages=2, _urlCap=1000)
c.GetallUrls()
'''
class Batch():
    def __init__(self, _sourceFile=None, _gapInSecond=0, _maxPages=500, _urlCap=1000, _fromRow=0, _colleges=-1):
        self.sourceFile=_sourceFile
        self.gapInSecond=_gapInSecond
        self.maxPages=_maxPages
        self.urlCap=_urlCap
        self.fromRow=_fromRow
        self.colleges=_colleges
        
    def GetMasterFile(self, file='C:\\Coding\\utility_git\\Notebooks\\training\\ContentAqusition_All.csv'):
        df_urls=pd.read_csv(self.sourceFile, names=['collegeId','url', 'name', 'filename','urlfile', 'urls', 'visitedurls',\
                                         'rejectedurls', 'filesizeinMB', 'randint'], sep=',', skiprows=1)
        r,l=df_urls.shape
        fromIndx,toIndx=0,0
        
        if self.fromRow>=r:
            fromIndx=r-1
        elif self.fromRow<0 or self.fromRow==None:
            fromIndx=0
        else:
            fromIndx=self.fromRow
            
        if self.colleges<=0 or self.colleges==None:
            toIndx=r
        else:
            toIndx=r if self.colleges+fromIndx>r else fromIndx+self.colleges
             
        return df_urls.iloc[fromIndx:toIndx]  

    def Run(self):
        df_urls=self.GetMasterFile() 
        k=0 
        ls_sum=[]
        n=0  
         
        for _, i in enumerate(df_urls.values): 
            collegeid,url, name, filename, urlfile, urls, visitedurls, rejectedurls,\
            filesizeinMB, randint=i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9] 
           
            try:
                c=CC.CollegeCrawl(name, url,urlfile, filename, _gapInSecond=self.gapInSecond, \
                                  _maxPages=self.maxPages, _capUrls=self.urlCap)
                c.GetallUrls()
                summary=c.Show_Summaries(show=False)
            
                if len(c.files)>0:
                    c.Merge(isRemoveRawFile=True)
                else:
                    print('({}) {} : No files writen'.format(c.collegeName, c.rootUrl))

                try:
                    dataFile=path.join(c.saveToFolder, filename)
                    fileSizeinMB=path.getsize(dataFile)/1024/1024
                except: 
                    fileSizeinMB=0.0

                zero=summary[summary['status_code']=='0']['url'].sum()
                t00=summary[summary['status_code']=='200']['url'].sum()
                full=summary['url'].sum()
                ls_sum.append([collegeid, full,  t00, full-zero-t00, "{0:.2f}".format(fileSizeinMB)])  
            except:
                print('Error: {} --{}  '.format(urlfile,name))
                pass 
        if ls_sum:
            self.UpdateStats(ls_sum)
        else:
            print('No college data collected.')

    def UpdateStats(self, ls_sum):
        df_sum=pd.DataFrame(ls_sum, columns=['collegeid', 'urls', 'visitedurls', 'rejectedurls', 'filesizeinMB'])    
        df_sum.to_csv(path.join(os.getcwd(), 'StagingFile.csv'), index = False)
        #print(df_sum)
        #print('#College Collected: {}'.len(ls_sum))
'''
b=Batch('C:\\Coding\\utility_git\\Notebooks\\training\\contentAqusition_Jason.csv',_maxPages=5, _fromRow=111, _colleges=3)
dff=b.Run()
'''    

In [ ]:
if __name__ == "__main__":  # confirms that the code is under main function
    folder='C:\\Coding\\utility_git\\Notebooks\\training\\' 
    files=[ path.join(folder,f) for f in listdir(folder) if re.match(r'^contentAqusition', f)]
     
    processes = []  
 
    for f in files: 
        b=Batch(f,_maxPages=250, _fromRow=-1, _colleges=-1)
        process = Process(target=b.Run())
        processes.append(process)
        process.start()
 
    for process in processes:
        process.join()

In [ ]:
folder='C:\\Coding\\utility_git\\Notebooks\\training\\' 
files=[ path.join(folder,f) for f in listdir(folder) if re.match(r'^contentAqusition', f) and 'All' not in f]